## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 4)
# trainLabel = trainL
print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=True)


In [3]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    model = Sequential() 
    model.add(Conv1D(filters = 32, kernel_size = 7, input_shape = (trainData.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 32, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(4, activation = "softmax"))
    
    adam = Adam(lr = 0.001)
    model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
    
    return model


batch_S = [60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
parameter_grid = dict(batch_size=batch_S)
model = KerasClassifier(build_fn = create_model, epochs=80, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, cv=kf, verbose = 1)
grid_result = grid.fit(trainData, trainLabel)

print('Finish training!')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Epoch 1/80
8120/8120 [==============================] - 14s 2ms/step - loss: 0.9861 - acc: 0.5840
Epoch 2/80
8120/8120 [==============================] - 7s 870us/step - loss: 0.8661 - acc: 0.6155
Epoch 3/80
8120/8120 [==============================] - 8s 936us/step - loss: 0.8092 - acc: 0.6555
Epoch 4/80
8120/8120 [==============================] - 7s 905us/step - loss: 0.7102 - acc: 0.7071
Epoch 5/80
8120/8120 [==============================] - 7s 915us/step - loss: 0.6612 - acc: 0.7325
Epoch 6/80
8120/8120 [==============================] - 8s 925us/step - loss: 0.6271 - acc: 0.7559
Epoch 7/80
8120/8120 [==============================] - 8s 928us/step - loss: 0.6120 - acc: 0.7559
Epoch 8/80
8120/8120 [==============================] - 7s 905us/step - loss: 0.5870 - acc: 0.7671
Epoch 9/80
8120/8120 [==============================] - 7s 903us/step - loss: 0.5606 - acc: 0.7794
Epoch 10/80
8120/8120 [==========================

8121/8121 [==============================] - 8s 1000us/step - loss: 0.9970 - acc: 0.5684
Epoch 2/80
8121/8121 [==============================] - 8s 935us/step - loss: 0.8727 - acc: 0.5987
Epoch 3/80
8121/8121 [==============================] - 8s 936us/step - loss: 0.7782 - acc: 0.6657
Epoch 4/80
8121/8121 [==============================] - 8s 930us/step - loss: 0.6906 - acc: 0.7144
Epoch 5/80
8121/8121 [==============================] - 8s 932us/step - loss: 0.6478 - acc: 0.7428
Epoch 6/80
8121/8121 [==============================] - 8s 941us/step - loss: 0.6254 - acc: 0.7513
Epoch 7/80
8121/8121 [==============================] - 7s 918us/step - loss: 0.5936 - acc: 0.7617
Epoch 8/80
8121/8121 [==============================] - 8s 957us/step - loss: 0.5657 - acc: 0.7765
Epoch 9/80
8121/8121 [==============================] - 8s 960us/step - loss: 0.5307 - acc: 0.7945
Epoch 10/80
8121/8121 [==============================] - 7s 919us/step - loss: 0.5210 - acc: 0.8041
Epoch 11/80
8121/81

8121/8121 [==============================] - 8s 1ms/step - loss: 1.0064 - acc: 0.5792
Epoch 2/80
8121/8121 [==============================] - 7s 907us/step - loss: 0.9057 - acc: 0.5890
Epoch 3/80
8121/8121 [==============================] - 7s 874us/step - loss: 0.8358 - acc: 0.6180
Epoch 4/80
8121/8121 [==============================] - 7s 904us/step - loss: 0.7436 - acc: 0.6806
Epoch 5/80
8121/8121 [==============================] - 7s 878us/step - loss: 0.6667 - acc: 0.7255
Epoch 6/80
8121/8121 [==============================] - 7s 879us/step - loss: 0.6423 - acc: 0.7369
Epoch 7/80
8121/8121 [==============================] - 7s 876us/step - loss: 0.5925 - acc: 0.7625
Epoch 8/80
8121/8121 [==============================] - 7s 896us/step - loss: 0.5876 - acc: 0.7652
Epoch 9/80
8121/8121 [==============================] - 7s 908us/step - loss: 0.5580 - acc: 0.7775
Epoch 10/80
8121/8121 [==============================] - 7s 876us/step - loss: 0.5354 - acc: 0.7909
Epoch 11/80
8121/8121 

Epoch 2/80
8121/8121 [==============================] - 7s 877us/step - loss: 0.8723 - acc: 0.6104
Epoch 3/80
8121/8121 [==============================] - 7s 880us/step - loss: 0.7599 - acc: 0.6743
Epoch 4/80
8121/8121 [==============================] - 7s 908us/step - loss: 0.6878 - acc: 0.7175
Epoch 5/80
8121/8121 [==============================] - 7s 893us/step - loss: 0.6382 - acc: 0.7394
Epoch 6/80
8121/8121 [==============================] - 7s 909us/step - loss: 0.5949 - acc: 0.7703
Epoch 7/80
8121/8121 [==============================] - 7s 914us/step - loss: 0.5562 - acc: 0.7913
Epoch 8/80
8121/8121 [==============================] - 7s 894us/step - loss: 0.5377 - acc: 0.7969
Epoch 9/80
8121/8121 [==============================] - 7s 890us/step - loss: 0.5063 - acc: 0.8158
Epoch 10/80
8121/8121 [==============================] - 7s 900us/step - loss: 0.5055 - acc: 0.8178
Epoch 11/80
8121/8121 [==============================] - 7s 893us/step - loss: 0.4806 - acc: 0.8223
Epoch 12

8121/8121 [==============================] - 7s 904us/step - loss: 0.8841 - acc: 0.6005
Epoch 3/80
8121/8121 [==============================] - 7s 896us/step - loss: 0.8074 - acc: 0.6521
Epoch 4/80
8121/8121 [==============================] - 7s 913us/step - loss: 0.7274 - acc: 0.6979
Epoch 5/80
8121/8121 [==============================] - 7s 876us/step - loss: 0.6684 - acc: 0.7242
Epoch 6/80
8121/8121 [==============================] - 7s 892us/step - loss: 0.6177 - acc: 0.7585
Epoch 7/80
8121/8121 [==============================] - 7s 905us/step - loss: 0.5992 - acc: 0.7633
Epoch 8/80
8121/8121 [==============================] - 7s 895us/step - loss: 0.5541 - acc: 0.7883
Epoch 9/80
8121/8121 [==============================] - 8s 926us/step - loss: 0.5382 - acc: 0.7998
Epoch 10/80
8121/8121 [==============================] - 8s 925us/step - loss: 0.5234 - acc: 0.8074
Epoch 11/80
8121/8121 [==============================] - 7s 903us/step - loss: 0.4965 - acc: 0.8137
Epoch 12/80
8121/81

...

8121/8121 [==============================] - 8s 927us/step - loss: 0.4208 - acc: 0.8421
Epoch 24/80
8121/8121 [==============================] - 8s 929us/step - loss: 0.3994 - acc: 0.8557
Epoch 25/80
8121/8121 [==============================] - 7s 908us/step - loss: 0.4041 - acc: 0.8511
Epoch 26/80
8121/8121 [==============================] - 7s 907us/step - loss: 0.4031 - acc: 0.8501
Epoch 27/80
8121/8121 [==============================] - 7s 909us/step - loss: 0.3819 - acc: 0.8562
Epoch 28/80
8121/8121 [==============================] - 7s 902us/step - loss: 0.3800 - acc: 0.8620
Epoch 29/80
8121/8121 [==============================] - 7s 906us/step - loss: 0.3874 - acc: 0.8572
Epoch 30/80
8121/8121 [==============================] - 7s 907us/step - loss: 0.3824 - acc: 0.8584
Epoch 31/80
8121/8121 [==============================] - 7s 906us/step - loss: 0.3724 - acc: 0.8642
Epoch 32/80
8121/8121 [==============================] - 7s 905us/step - loss: 0.3794 - acc: 0.8618
Epoch 33/80


Epoch 24/80
8121/8121 [==============================] - 8s 940us/step - loss: 0.4020 - acc: 0.8536
Epoch 25/80
8121/8121 [==============================] - 7s 907us/step - loss: 0.3960 - acc: 0.8546
Epoch 26/80
8121/8121 [==============================] - 7s 905us/step - loss: 0.3997 - acc: 0.8543
Epoch 27/80
8121/8121 [==============================] - 7s 923us/step - loss: 0.3994 - acc: 0.8601
Epoch 28/80
8121/8121 [==============================] - 8s 933us/step - loss: 0.3940 - acc: 0.8570
Epoch 29/80
8121/8121 [==============================] - 7s 906us/step - loss: 0.3824 - acc: 0.8652
Epoch 30/80
8121/8121 [==============================] - 7s 907us/step - loss: 0.3819 - acc: 0.8623
Epoch 31/80
8121/8121 [==============================] - 7s 913us/step - loss: 0.3728 - acc: 0.8663
Epoch 32/80
8121/8121 [==============================] - 7s 907us/step - loss: 0.3688 - acc: 0.8655
Epoch 33/80
8121/8121 [==============================] - 7s 908us/step - loss: 0.3693 - acc: 0.8671


8121/8121 [==============================] - 8s 927us/step - loss: 0.3980 - acc: 0.8533
Epoch 25/80
8121/8121 [==============================] - 8s 939us/step - loss: 0.3967 - acc: 0.8548
Epoch 26/80
8121/8121 [==============================] - 7s 909us/step - loss: 0.3770 - acc: 0.8620
Epoch 27/80
8121/8121 [==============================] - 7s 907us/step - loss: 0.3833 - acc: 0.8570
Epoch 28/80
8121/8121 [==============================] - 7s 909us/step - loss: 0.3659 - acc: 0.8642
Epoch 29/80
8121/8121 [==============================] - 7s 910us/step - loss: 0.3601 - acc: 0.8711
Epoch 30/80
8121/8121 [==============================] - 8s 941us/step - loss: 0.3618 - acc: 0.8631
Epoch 31/80
8121/8121 [==============================] - 7s 911us/step - loss: 0.3578 - acc: 0.8729
Epoch 32/80
8121/8121 [==============================] - 7s 909us/step - loss: 0.3519 - acc: 0.8724
Epoch 33/80
8121/8121 [==============================] - 7s 911us/step - loss: 0.3535 - acc: 0.8694
Epoch 34/80


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 499.2min finished


Epoch 1/80
10151/10151 [==============================] - 16s 2ms/step - loss: 0.9674 - acc: 0.5754
Epoch 2/80
10151/10151 [==============================] - 9s 911us/step - loss: 0.8430 - acc: 0.6225
Epoch 3/80
10151/10151 [==============================] - 9s 910us/step - loss: 0.7660 - acc: 0.6738
Epoch 4/80
10151/10151 [==============================] - 9s 919us/step - loss: 0.6968 - acc: 0.7110
Epoch 5/80
10151/10151 [==============================] - 9s 919us/step - loss: 0.6534 - acc: 0.7329
Epoch 6/80
10151/10151 [==============================] - 9s 935us/step - loss: 0.6221 - acc: 0.7457
Epoch 7/80
10151/10151 [==============================] - 10s 939us/step - loss: 0.6003 - acc: 0.7646
Epoch 8/80
10151/10151 [==============================] - 9s 916us/step - loss: 0.5835 - acc: 0.7755
Epoch 9/80
10151/10151 [==============================] - 9s 922us/step - loss: 0.5424 - acc: 0.7914
Epoch 10/80
10151/10151 [==============================] - 9s 920us/step - loss: 0.5320 - a

In [4]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean:%f (STD:%f) with: %r" % (mean, stdev, param))

Mean:0.844350 (STD:0.005974) with: {'batch_size': 60}
Mean:0.842774 (STD:0.008817) with: {'batch_size': 61}
Mean:0.843858 (STD:0.008388) with: {'batch_size': 62}
Mean:0.842183 (STD:0.006005) with: {'batch_size': 63}
Mean:0.845434 (STD:0.006181) with: {'batch_size': 64}
Mean:0.840213 (STD:0.005540) with: {'batch_size': 65}
Mean:0.848980 (STD:0.007734) with: {'batch_size': 66}
Mean:0.844843 (STD:0.005493) with: {'batch_size': 67}
Mean:0.844153 (STD:0.004493) with: {'batch_size': 68}
Mean:0.840213 (STD:0.003276) with: {'batch_size': 69}


In [5]:
grid_result.best_score_

0.8489803971592258

In [6]:
grid_result.best_params_

{'batch_size': 66}